Check this out https://www.youtube.com/watch?v=Jl-Ye38qkRc

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from modules.prepro import prepro
from modules.functions import bound_logistic, exec_timer
%matplotlib qt

In [2]:
# run prepro (does nothing if there is nothing to do)
dataroot = Path("../data/test/4afc")
prepro(dataroot)

# load the data
data = pd.read_csv(f"{dataroot}/4afc_all.csv")

# select example dataset
name = "eva"
condition = 'high'

# get example x and y data
x = data.what[(data.cat == condition) & (data.subj == name)]
y = data.prop[(data.cat == condition) & (data.subj == name)]

WARNING The directories already exist! Nothing will be done unless you delete them!
SUCCESS Output file created or exists already!


In [3]:
# make a logistic function 
logistic = bound_logistic(0.25, 1)

# fit the function
popt, pcov = curve_fit(logistic, x, y, method="trf", maxfev=50000)

# make the model curve with higher resolution
x_fit = np.arange(0, 0.12, 0.001)
y_fit = logistic(x_fit, *popt)

# plot the data and the model
plt.plot(x, y, 'o', label=name)
plt.plot(x_fit, y_fit)

SyntaxError: invalid syntax (2009004946.py, line 3)

In [ ]:
# find nearest value in an array
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

# plot curves for 3 in one condition
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

condition = 'high'
colors = ['blue', 'green', 'red']

for i, name in enumerate(np.unique(data.subj)[:3]):

    # get data for this name
    x = np.asanyarray(data.what[(data.cat == condition) & (data.subj == name)], dtype=np.float64)
    y = np.asanyarray(data.prop[(data.cat == condition) & (data.subj == name)], dtype=np.float64)

    # fit the logistic function
    popt, pcov = curve_fit(logistic, x, y, method="trf", maxfev=50000)

    # make the model curve
    x_fit = np.arange(0, 0.12, 0.001)
    y_fit = logistic(x_fit, *popt)

    # plot
    ax.plot(x, y, 'o', label=name, c=colors[i],)
    ax.plot(
        x_fit, 
        y_fit, 
        '-',  
        c=colors[i],
    )

    thresh = x_fit[y_fit == find_nearest(y_fit,0.625)][0]
    threshy = y_fit[y_fit == find_nearest(y_fit,0.625)][0]

    ax.plot([thresh, thresh], [0.25, threshy], lw=1, label=f"det. thresh = {np.round(thresh, 2)}", c=colors[i],)

ax.set_ylabel("prop. correct")
ax.set_xlabel('duration visible [s]')
ax.set_ylim(0.25, 1)
ax.legend()
plt.show()

In [ ]:
# iterate through all in both categories and collect thresholds
conds = []
threshs = []
for condition in np.unique(data.cat):
    for i, name in enumerate(np.unique(data.subj)):

        # get data for this name
        x = np.asanyarray(data.what[(data.cat == condition) & (data.subj == name)], dtype=np.float64)
        y = np.asanyarray(data.prop[(data.cat == condition) & (data.subj == name)], dtype=np.float64)

        # fit the logistic function
        popt, pcov = curve_fit(logistic, x, y, method="trf", maxfev=50000)

        # make the model curve
        x_fit = np.arange(0, 0.12, 0.001)
        y_fit = logistic(x_fit, *popt)

        thresh = x_fit[y_fit == find_nearest(y_fit,0.625)][0]
        threshy = y_fit[y_fit == find_nearest(y_fit,0.625)][0]

        conds.append(condition)
        threshs.append(thresh)

# make a dataframe 
dthresh = pd.DataFrame(np.asarray([np.asarray(conds), np.asarray(threshs)]).T, columns=('cond', 'thresh'))

In [ ]:
# plot detection thresholds for two conditions

fig, ax = plt.subplots()

for n, cond in enumerate(np.unique(dthresh.cond)):
    ax.boxplot(dthresh.thresh[dthresh.cond == cond], positions=[n+1], notch=True)

TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''